# GANNs with friends - Google Colab worker

Run this notebook to participate in distributed GAN training using Google Colab's free GPU.

## Setup environment

Install required packages and clone the repository.

In [ ]:
# Clone repository if not already present
import os
if not os.path.exists('GANNs-with-friends'):
    !git clone https://github.com/gperdrizet/GANNs-with-friends.git
    %cd GANNs-with-friends
else:
    %cd GANNs-with-friends
    !git pull

In [ ]:
# Install dependencies
# Note: You may see warnings about numpy version conflicts with Colab's pre-installed
# packages (jax, opencv, etc.). These can be safely ignored - we don't use those packages.
!pip install --quiet -r requirements.txt

In [ ]:
# Check GPU availability
# If this shows "No GPU", go to Runtime > Change runtime type > T4 GPU
import torch
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('No GPU detected! Enable GPU in Runtime > Change runtime type')

## Download CelebA dataset

Downloads the CelebA face dataset from Hugging Face (~1.3 GB). This only runs once.

In [ ]:
!python scripts/download_celeba.py

## Configure worker

Copy the template config and customize your settings.

**Required:** Database credentials (get these from the training coordinator)

**Optional but recommended:**
- Your name (shown on the dashboard leaderboard)
- Batch size (tune for your GPU memory - Colab T4 can usually handle 64)

In [ ]:
# Copy template if config does not exist
if not os.path.exists('config.yaml'):
    !cp config.yaml.template config.yaml
    print('Created config.yaml from template')
    print('Edit config.yaml with your database credentials before continuing')
else:
    print('config.yaml already exists')

In [ ]:
# Display current config
!cat config.yaml

### Edit config.yaml

Use the file browser in the left sidebar to open and edit `config.yaml`.

**Database section** (required - get these from the training coordinator):
```yaml
database:
  host: YOUR_DATABASE_HOST
  user: YOUR_DATABASE_USER
  password: YOUR_DATABASE_PASSWORD
```

**Worker section** (optional customization):
```yaml
worker:
  name: YourName          # Shows on dashboard leaderboard
  batch_size: 64          # Colab T4 can handle 64; use 32 if you get memory errors
```

## Run worker

Start the training worker. It will:
1. Connect to the database and register your worker
2. Download the current model weights
3. Process work units and upload gradients
4. Repeat until training completes or you stop it

Your progress will appear on the coordinator's dashboard.

In [ ]:
!python src/worker.py